<a href="https://colab.research.google.com/github/in-c0/AgroPest-12-YOLOv11/blob/main/AgroPest_12_YOLOv11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip -q install ultralytics==8.3.28 kaggle transformers==4.44.2 timm==1.0.9 torchmetrics==1.4.1 scikit-learn==1.5.2 opencv-python==4.10.0.84

import os, json, shutil, zipfile, glob, random, math, itertools, time
from pathlib import Path
import numpy as np
import pandas as pd
import cv2
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.2/881.2 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 118.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 108.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.2 which is incompatible.


'cuda'

In [3]:
# Configure kaggle credentials
from google.colab import files
if not os.path.exists('/root/.kaggle/kaggle.json'):
    print("🔑 Upload your kaggle.json")
    files.upload()
    os.makedirs('/root/.kaggle', exist_ok=True)
    shutil.move('kaggle.json', '/root/.kaggle/kaggle.json')
    os.chmod('/root/.kaggle/kaggle.json', 0o600)

🔑 Upload your kaggle.json


Saving kaggle.json to kaggle.json


In [4]:
!rm -rf /content/datasets/

In [5]:
# Download dataset
import kagglehub
path = kagglehub.dataset_download("rupankarmajumdar/crop-pests-dataset")

print("Path to dataset files:", path)
import os, shutil
src = "/root/.cache/kagglehub/datasets/rupankarmajumdar/crop-pests-dataset/versions/2"
dst = "/content/datasets/agropest"
if os.path.exists(dst):
    shutil.rmtree(dst)
shutil.copytree(src, dst)
!ls -R "$dst" | head -n 5


100%|██████████| 537M/537M [00:13<00:00, 40.3MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/rupankarmajumdar/crop-pests-dataset/versions/2
/content/datasets/agropest:
data.yaml
test
train
valid


In [6]:
# YOLOv11n Baseline
from ultralytics import YOLO
model = YOLO('yolo11n.pt')
results = model.train(
    data=str(dst + "/data.yaml"),
    imgsz=640,
    epochs=50,
    batch=16,
    device=0 if device=='cuda' else 'cpu',
    workers=8,
    cos_lr=True,
    optimizer='auto',
    amp=True,
    seed=42,
)

val_metrics = model.val(split='val', imgsz=640, device=0 if device=='cuda' else 'cpu')
test_metrics = model.val(split='test', imgsz=640, device=0 if device=='cuda' else 'cpu')
print("VAL metrics:", val_metrics.results_dict)
print("TEST metrics:", test_metrics.results_dict)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 156MB/s]


New https://pypi.org/project/ultralytics/8.3.218 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.28 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/datasets/agropest/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save

100%|██████████| 755k/755k [00:00<00:00, 39.4MB/s]


Overriding model.yaml nc=80 with nc=12

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytic

...

                   all       1095       1341      0.814      0.738      0.771      0.463
                  Ants         96        178      0.808      0.674      0.728      0.279
                  Bees         99        110      0.881      0.827      0.863      0.396
               Beetles         89        100      0.732      0.682      0.667      0.319
          Caterpillars         77        139      0.696       0.41      0.463      0.236
            Earthworms         53         72      0.664      0.472      0.469      0.271
               Earwigs         91        104      0.782       0.75      0.752      0.428
          Grasshoppers         98        102      0.718      0.716      0.764      0.415
                 Moths        100        101       0.99      0.941      0.979      0.753
                 Slugs         77         91      0.655      0.604       0.71      0.439
                Snails         99        107      0.932      0.935      0.957      0.675
                 Wasp

val: Scanning /content/datasets/agropest/test/labels... 546 images, 0 backgrounds, 0 corrupt: 100%|██████████| 546/546 [00:00<00:00, 2270.70it/s]

val: New cache created: /content/datasets/agropest/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:06<00:00,  5.07it/s]


                   all        546        689      0.846      0.708      0.778      0.469
                  Ants         54         87      0.955      0.727      0.814      0.324
                  Bees         40         44      0.874      0.864      0.922      0.453
               Beetles         41         44      0.757      0.636      0.717      0.411
          Caterpillars         46         93      0.749       0.43      0.611       0.31
            Earthworms         27         40      0.651      0.425      0.458      0.235
               Earwigs         59         73      0.859      0.658      0.745      0.414
          Grasshoppers         38         55      0.784      0.545      0.617      0.302
                 Moths         47         47      0.989      0.894      0.964      0.805
                 Slugs         46         51      0.754      0.549      0.672      0.409
                Snails         44         50       0.86       0.84      0.847      0.639
                 Wasp

In [16]:
from pathlib import Path
import random

metrics = model.val(data="/content/datasets/agropest/data.yaml", split="test")


print("Mean Precision:", metrics.box.mp)
print("Mean Recall:", metrics.box.mr)
print("mAP@0.5:", metrics.box.map50)
print("mAP@0.5:0.95:", metrics.box.map)

for i, cname in enumerate(model.names.values()):
    print(f"{cname:15s} | P: {metrics.box.p[i]:.3f} | R: {metrics.box.r[i]:.3f} | AP50: {metrics.box.ap50[i]:.3f} | AP: {metrics.box.ap[i]:.3f}")



Ultralytics 8.3.28 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)


val: Scanning /content/datasets/agropest/test/labels.cache... 546 images, 0 backgrounds, 0 corrupt: 100%|██████████| 546/546 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [00:08<00:00,  4.12it/s]


                   all        546        689      0.846      0.708      0.778      0.469
                  Ants         54         87      0.955      0.727      0.814      0.324
                  Bees         40         44      0.874      0.864      0.922      0.453
               Beetles         41         44      0.757      0.636      0.717      0.411
          Caterpillars         46         93      0.749       0.43      0.611       0.31
            Earthworms         27         40      0.651      0.425      0.458      0.235
               Earwigs         59         73      0.859      0.658      0.745      0.414
          Grasshoppers         38         55      0.784      0.545      0.617      0.302
                 Moths         47         47      0.989      0.894      0.964      0.805
                 Slugs         46         51      0.754      0.549      0.672      0.409
                Snails         44         50       0.86       0.84      0.847      0.639
                 Wasp